In [5]:

import os
os.environ["OMP_NUM_THREADS"] = "32"
from graph_tool.all import *
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.covariance import LedoitWolf, OAS
import matplotlib.pyplot as plt
import matplotlib.cm as mpl
import seaborn as sns
import statsmodels.api as sm
from multipy.fdr import qvalue
from multipy.fdr import lsu

import dill

In [6]:
# Loading blocks...
def load_blocks(blocks):
    with open (blocks, "rb") as fh:
        bs = dill.load(fh)[0:6]
    return bs
# For each unique element in x, find its first apperance in x
def first_occurrence(x):
    _, idx = np.unique(x, return_index=True)
    return idx

def labelVertices(state):
    g = state.g
    g.vp.level_0 = g.new_vertex_property("double", state.get_bs()[0])
    first = first_occurrence(np.array([g.vp.level_0.a]))
    state.g.vp.labels = g.new_vp("string", [str(int(x)) if i in first else "" for i, x in enumerate(g.vp.level_0.a)])
    return state

In [8]:
g_path = '../../SBM/snakemake/cache/trimmed_graph/fdr-1e-3/layered/'
tissues = ['head', 'body']
conditions = ['hs', 'ctrl']
graphs = {f'{tissue}':load_graph(g_path + f'{tissue}.xml.gz') for tissue in tissues}
b_path = '../../SBM/snakemake/cache/MCMC/blocks/fdr-1e-3/layered/'
blocks = {f'{tissue}':load_blocks(b_path + f'{tissue}.dill') for tissue in tissues}

labels = [f'{tissue}-{condition}' for tissue in tissues for condition in conditions]

In [9]:
cond_graphs = {}
for t in tissues:
    g = graphs[t]
    ds = g.ep.dataset
    for c in conditions:
        l = f'{t}-{c}'
        print(l)
        u = GraphView(g, efilt=lambda e: ds[e] == c)
        u = Graph(u, prune=True)  
        cond_graphs[l] = u  

head-hs
head-ctrl
body-hs
body-ctrl


In [10]:
state = {}
for t in tissues:
    for c in conditions:
        l = f'{t}-{c}'
        print(l)
        g = cond_graphs[l]
        bs = blocks[t]
        # Arctan transform on correlations...
        corr = g.edge_properties["spearman"]
        g.ep.weight = g.new_edge_property("double", (2*np.arctanh(corr.a)))

        # Creating nested block model...
        state[l] = NestedBlockState(g, bs=bs,
                                    state_args=dict(recs=[g.ep.weight],
                                                    rec_types=["real-normal"]))

head-hs
head-ctrl
body-hs
body-ctrl


In [ ]:
pos = {}
for l in labels:
    s = state[l]
    g = cond_graphs[l]
    g.vp.level_0 = g.new_vertex_property("double", s.get_bs()[0])
    print(l)
    pos_t, twat, tpos = s.draw( eorder=g.ep.weight,
            # edge_pen_width = 0,
            edge_color=prop_to_size(g.ep.weight, mi=-4, ma=4, power=1, log=False),
            ecmap=(mpl.inferno, .6), 
            edge_gradient=[], 
            vertex_size = 4,
            hvertex_size = 25,
            hedge_pen_width = 3,
            vertex_color = g.vp.level_0,
            vertex_fill_color = g.vp.level_0,
            subsample_edges = 30000,
            output = "../../tmp/" + l + ".png", 
            output_size=(2000, 2000))
    pos[l] = pos_t

head-hs
head-ctrl
body-hs
body-ctrl


In [20]:
pos = {}
for t in tissues:
    c = 'ctrl'
    l = f'{t}-{c}'
    s = state[l]
    g = cond_graphs[l]
    labelVertices(s)
    print(l)
    pos_t, twat, tpos = s.draw( eorder=s.g.ep.weight,
            # edge_pen_width = 0,
            edge_color=prop_to_size(s.g.ep.weight, mi=-4, ma=4, power=1, log=False),
            ecmap=(mpl.inferno, .6), 
            edge_gradient=[], 
            vertex_size = 10,
            vertex_text = s.g.vp.labels,
            vertex_text_position='centered',
            hvertex_size = 25,
            hedge_pen_width = 3,
            vertex_color = s.g.vp.level_0,
            vertex_fill_color = s.g.vp.level_0,
            subsample_edges = 20000,
            output = "../../tmp/" + l + ".png", 
            output_size=(2000, 2000))
    g.vp.pos = g.copy_property(pos_t)
    pos[t] = pos_t
for t in tissues:
    c = 'hs'
    l = f'{t}-{c}'
    s = state[l]
    g = cond_graphs[l]
    g.vp.pos = g.copy_property(pos[t])
    labelVertices(s)
    print(l)
    s.draw( eorder=s.g.ep.weight,
            # edge_pen_width = 0,
            pos = g.vp.pos, 
            edge_color=prop_to_size(s.g.ep.weight, mi=-4, ma=4, power=1, log=False),
            ecmap=(mpl.inferno, .6), 
            edge_gradient=[], 
            vertex_size = 10,
            vertex_text = s.g.vp.labels,
            vertex_text_position='centered',
            hvertex_size = 25,
            hedge_pen_width = 3,
            vertex_color = s.g.vp.level_0,
            vertex_fill_color = s.g.vp.level_0,
            subsample_edges = 20000,
            output = "../../tmp/" + l + ".png", 
            output_size=(2000, 2000)) 

head-ctrl
body-ctrl
head-hs
body-hs


In [23]:
# Save cond graphs in cache
for l in labels:
    g = cond_graphs[l]
    g.save(f'../../cache/cond_graph-{l}.xml.gz')